In [34]:
# IMPORT LIBRARIES

# Processing
import librosa
import librosa.display
import numpy as np
import random
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import IPython.display as ipd

# Files
import os
import joblib
import pickle

# Machine Learning
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
import tensorflow as tf

# ####### TEST ####### 
# Scipy
from scipy import signal
from scipy.io import wavfile

In [35]:
# AUDIO_DATA_PATH = 'data/'
GPATH = '/content/drive/My Drive/Master/Asignaturas/2 Cuatrimestre/Proyecto/Code/'
# LPATH_DATA = 'SpeechEmotionRecognition/data/processed/'
# LPATH_AUGMENTED = GPATH + AUDIO_DATA_PATH + 'RAVDESS/augmented'

SAVEE_PATH = GPATH + 'data/SAVEE/'
TESS_PATH = GPATH + 'data/TESS/'

# Maps
EMOTION_MAP = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
INTENSITY_MAP = {1:'normal', 2:'strong'}

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se leen los datasets

In [37]:
# Leemos SAVEE

emotion=[]
path = []

for dir in os.listdir(SAVEE_PATH):
  path_dir = os.listdir(SAVEE_PATH + dir) # todos los archivos de audios asociados a un directorio
  for filename in path_dir:
    str_path = SAVEE_PATH + dir + '/' + str(filename)
    path.append(str_path)
    if(filename[0]=='a'):
        emotion.append("anger")
    elif(filename[0]=='h'):
        emotion.append("happy")
    elif(filename[0]=='f'):
        emotion.append("fear")
    elif(filename[0]=='d'):
        emotion.append("disgust")
    elif(filename[0]=='n'):
        emotion.append('neutral')
    elif(filename[0]=='s'):
        if(filename[1]=='a'):
            emotion.append("sad")
        elif(filename[1]=='u'):
            emotion.append("surprise")


SAVEE_df = pd.DataFrame(columns=['emotion', 'path'])
SAVEE_df['emotion'] = emotion
SAVEE_df['path'] = path

In [38]:
SAVEE_df.head()

,emotion,path
0,anger,/content/drive/My Drive/Master/Asignaturas/2 C...
1,anger,/content/drive/My Drive/Master/Asignaturas/2 C...
2,neutral,/content/drive/My Drive/Master/Asignaturas/2 C...
3,neutral,/content/drive/My Drive/Master/Asignaturas/2 C...
4,neutral,/content/drive/My Drive/Master/Asignaturas/2 C...


In [39]:
# Leemos TESS

emotion=[]
path = []

for dir in os.listdir(TESS_PATH):
  path_dir = os.listdir(TESS_PATH + dir) # todos los archivos de audios asociados a un directorio
  label = dir.split('_')[1]
  for filename in path_dir:
    str_path = TESS_PATH + dir + '/' + str(filename)
    path.append(str_path)
    emotion.append(label)


TESS_df = pd.DataFrame(columns=['emotion', 'path'])
TESS_df['emotion'] = emotion
TESS_df['path'] = path

TESS_df.head()

,emotion,path
0,Sad,/content/drive/My Drive/Master/Asignaturas/2 C...
1,Sad,/content/drive/My Drive/Master/Asignaturas/2 C...
2,Sad,/content/drive/My Drive/Master/Asignaturas/2 C...
3,Sad,/content/drive/My Drive/Master/Asignaturas/2 C...
4,Sad,/content/drive/My Drive/Master/Asignaturas/2 C...


Se procesan y se juntan

In [42]:
# dataset = pd.concat([n_dataset,pd.DataFrame(dataset['feature'].values.tolist())],axis=1)
import sys  
sys.path.insert(0, GPATH + '/SpeechEmotionRecognition')

In [47]:
from src.MFCC import MFCC

featuresMFCC = MFCC(TESS_df)
featuresMFCC.read_features_dataAugmentation()

  0%|          | 7/2800 [00:03<18:54,  2.46it/s]

KeyboardInterrupt: ignored

Se entrenan